<a href="https://colab.research.google.com/github/Elish-Ab/AI_Mastery_10x_Week8_9/blob/main/notebooks/Task_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.4/568.4 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.0 MB/s eta 0:00:00


In [18]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras import layers
import mlflow
import mlflow.sklearn

In [13]:
# Load datasets
fraud_data = pd.read_csv('Fraud_Data.csv')
credit_data = pd.read_csv('creditcard.csv')


In [16]:
# Fraud_Data.csv
# Convert 'purchase_time' to datetime
fraud_data['purchase_time'] = pd.to_datetime(fraud_data['purchase_time'])

# Convert 'purchase_time' to datetime
fraud_data['purchase_time'] = pd.to_datetime(fraud_data['purchase_time'])

# Extract useful time-based features
fraud_data['hour_of_day'] = fraud_data['purchase_time'].dt.hour
fraud_data['day_of_week'] = fraud_data['purchase_time'].dt.weekday

# Drop columns not needed for training
X_fraud = fraud_data.drop(columns=['purchase_time', 'signup_time', 'class', 'user_id', 'device_id', 'ip_address'])

# One-hot encode categorical columns (e.g., 'source', 'browser', 'sex')
X_fraud = pd.get_dummies(X_fraud, columns=['source', 'browser', 'sex'], drop_first=True)

# Separate target variable
y_fraud = fraud_data['class']

# Split data
X_train_fraud, X_test_fraud, y_train_fraud, y_test_fraud = train_test_split(X_fraud, y_fraud, test_size=0.3, random_state=42)

# creditcard.csv
X_credit = credit_data.drop(columns=['Class'])
y_credit = credit_data['Class']
X_train_credit, X_test_credit, y_train_credit, y_test_credit = train_test_split(X_credit, y_credit, test_size=0.3, random_state=42)

In [21]:
def evaluate_model(model, X_test, y_test, name):
    y_pred = model.predict(X_test)
    print(f"{name} Report:\n", classification_report(y_test, y_pred))

In [20]:
# Start MLflow experiment tracking
mlflow.end_run()  # Ensure any previous run is closed
mlflow.start_run()

# Scaling data
scaler = StandardScaler()
X_train_fraud_scaled = scaler.fit_transform(X_train_fraud)
X_test_fraud_scaled = scaler.transform(X_test_fraud)
X_train_credit_scaled = scaler.fit_transform(X_train_credit)
X_test_credit_scaled = scaler.transform(X_test_credit)

Model training and Evaluation

In [26]:
# Function to evaluate models
def evaluate_model(model, X_test, y_test, name):
    y_pred = model.predict(X_test)
    print(f"{name} Report:\n", classification_report(y_test, y_pred))

# Ensure previous MLflow run is ended
mlflow.end_run()

# Start the main MLflow run
with mlflow.start_run() as parent_run:

    # Logistic Regression (Fraud Data)
    with mlflow.start_run(nested=True):
        logreg_fraud = LogisticRegression(class_weight='balanced')
        logreg_fraud.fit(X_train_fraud_scaled, y_train_fraud)
        evaluate_model(logreg_fraud, X_test_fraud_scaled, y_test_fraud, "Logistic Regression (Fraud Data)")
        mlflow.log_param("Model", "Logistic Regression (Fraud Data)")
        mlflow.log_metric("Accuracy_Fraud_Data", logreg_fraud.score(X_test_fraud_scaled, y_test_fraud))
        mlflow.sklearn.log_model(logreg_fraud, "logreg_fraud_model")

    # Logistic Regression (Credit Data)
    with mlflow.start_run(nested=True):
        logreg_credit = LogisticRegression(class_weight='balanced')
        logreg_credit.fit(X_train_credit_scaled, y_train_credit)
        evaluate_model(logreg_credit, X_test_credit_scaled, y_test_credit, "Logistic Regression (Credit Data)")
        mlflow.log_param("Model", "Logistic Regression (Credit Data)")
        mlflow.log_metric("Accuracy_Credit_Data", logreg_credit.score(X_test_credit_scaled, y_test_credit))
        mlflow.sklearn.log_model(logreg_credit, "logreg_credit_model")

    # Decision Tree (Fraud Data)
    with mlflow.start_run(nested=True):
        dtree_fraud = DecisionTreeClassifier(random_state=42)
        dtree_fraud.fit(X_train_fraud_scaled, y_train_fraud)
        evaluate_model(dtree_fraud, X_test_fraud_scaled, y_test_fraud, "Decision Tree (Fraud Data)")
        mlflow.log_param("Model", "Decision Tree (Fraud Data)")
        mlflow.log_metric("Accuracy_Fraud_Data_DTree", dtree_fraud.score(X_test_fraud_scaled, y_test_fraud))
        mlflow.sklearn.log_model(dtree_fraud, "dtree_fraud_model")

    # Random Forest (Fraud Data)
    with mlflow.start_run(nested=True):
        rf_fraud = RandomForestClassifier(random_state=42)
        rf_fraud.fit(X_train_fraud_scaled, y_train_fraud)
        evaluate_model(rf_fraud, X_test_fraud_scaled, y_test_fraud, "Random Forest (Fraud Data)")
        mlflow.log_param("Model", "Random Forest (Fraud Data)")
        mlflow.log_metric("Accuracy_Fraud_Data_RF", rf_fraud.score(X_test_fraud_scaled, y_test_fraud))
        mlflow.sklearn.log_model(rf_fraud, "rf_fraud_model")

    # Gradient Boosting (Fraud Data)
    with mlflow.start_run(nested=True):
        gb_fraud = GradientBoostingClassifier(random_state=42)
        gb_fraud.fit(X_train_fraud_scaled, y_train_fraud)
        evaluate_model(gb_fraud, X_test_fraud_scaled, y_test_fraud, "Gradient Boosting (Fraud Data)")
        mlflow.log_param("Model", "Gradient Boosting (Fraud Data)")
        mlflow.log_metric("Accuracy_Fraud_Data_GB", gb_fraud.score(X_test_fraud_scaled, y_test_fraud))
        mlflow.sklearn.log_model(gb_fraud, "gb_fraud_model")

    # Multi-Layer Perceptron (MLP) with TensorFlow (Fraud Data)
    with mlflow.start_run(nested=True):
        mlp_model = tf.keras.Sequential([
            layers.Dense(64, activation='relu', input_shape=(X_train_fraud_scaled.shape[1],)),
            layers.Dense(32, activation='relu'),
            layers.Dense(1, activation='sigmoid')
        ])

        mlp_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        mlp_model.fit(X_train_fraud_scaled, y_train_fraud, epochs=10, batch_size=32, validation_split=0.2)

        # Evaluate MLP model
        mlp_loss, mlp_accuracy = mlp_model.evaluate(X_test_fraud_scaled, y_test_fraud)
        mlflow.log_param("Model", "MLP (Fraud Data)")
        mlflow.log_metric("Accuracy_Fraud_Data_MLP", mlp_accuracy)

        # Log the MLP model using mlflow.tensorflow
        mlflow.tensorflow.log_model(mlp_model, "mlp_fraud_model")

# End MLflow parent run
mlflow.end_run()

Logistic Regression (Fraud Data) Report:
               precision    recall  f1-score   support

           0       0.91      0.54      0.68     41117
           1       0.10      0.50      0.17      4217

    accuracy                           0.54     45334
   macro avg       0.51      0.52      0.42     45334
weighted avg       0.84      0.54      0.63     45334



2024/10/22 06:54:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Logistic Regression (Credit Data) Report:
               precision    recall  f1-score   support

           0       1.00      0.97      0.99     85307
           1       0.06      0.93      0.10       136

    accuracy                           0.97     85443
   macro avg       0.53      0.95      0.55     85443
weighted avg       1.00      0.97      0.99     85443



2024/10/22 06:54:10 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Decision Tree (Fraud Data) Report:
               precision    recall  f1-score   support

           0       0.95      0.93      0.94     41117
           1       0.45      0.57      0.50      4217

    accuracy                           0.89     45334
   macro avg       0.70      0.75      0.72     45334
weighted avg       0.91      0.89      0.90     45334



...

2024/10/22 06:54:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Gradient Boosting (Fraud Data) Report:
               precision    recall  f1-score   support

           0       0.91      1.00      0.95     41117
           1       1.00      0.00      0.00      4217

    accuracy                           0.91     45334
   macro avg       0.95      0.50      0.48     45334
weighted avg       0.92      0.91      0.86     45334



2024/10/22 06:54:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
2645/2645 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.8951 - loss: 0.3329 - val_accuracy: 0.9076 - val_loss: 0.3078
Epoch 2/10
2645/2645 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - accuracy: 0.9052 - loss: 0.3127 - val_accuracy: 0.9076 - val_loss: 0.3059
Epoch 3/10
2645/2645 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.9066 - loss: 0.3077 - val_accuracy: 0.9076 - val_loss: 0.3050
Epoch 4/10
2645/2645 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9080 - loss: 0.3037 - val_accuracy: 0.9076 - val_loss: 0.3045
Epoch 5/10
2645/2645 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.9052 - loss: 0.3083 - val_accuracy: 0.9076 - val_loss: 0.3048
Epoch 6/10
2645/2645 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.9043 - loss: 0.3085 - val_accuracy: 0.9076 - val_loss: 0.3019
Epoch 7/10
2645/2645 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9068 - loss: 0.3003 - val_accuracy: 0.9077 - val_loss: 0.3011
Epoch 8/10
2645/2645 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9044 - loss: 0.3051 

2024/10/22 06:56:12 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2024/10/22 06:56:19 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
